# Hosting Capacity Deterministic Analysis

In [ ]:
import os, math, sys
import pandas as pd
import pandapower as pp
import numpy as np

from pandapower.file_io import from_json, to_json

main_dir = os.path.abspath(os.path.join(os.path.abspath(""), ".."))
data_dir = os.path.join(main_dir, 'Modified_116_LV_CSV')

if main_dir not in sys.path:
    sys.path.insert(0, main_dir)

import create_basic_network as cbn
import pp_simu

In [ ]:
net = from_json(os.path.join(main_dir, 'json_networks', 'full_load_network.json'))
print(net)
net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].describe().to_csv(os.path.join(data_dir, 'full_net.res_bus_3ph_describe.csv'))

profile_df, ds = cbn.create_data_source(data_dir)
ds.df.head(10)

In [ ]:
output_path = os.path.join(main_dir, 'hc_results', 'DET')
os.makedirs(output_path) if not os.path.exists(output_path) else None

### 1. PV-only scenario

In [ ]:
pv_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=True, ev=False)

pv_hc_results.to_csv(os.path.join(output_path, 'pv_hc_results.csv'))

### 2. EV-only scenario

In [ ]:
ev_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=False, ev=True)

ev_hc_results.to_csv(os.path.join(output_path, 'ev_hc_results.csv'))

### 3. PV+EV scenario

In [ ]:
pvev_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=True, ev=True)

pvev_hc_results.to_csv(os.path.join(output_path, 'pvev_hc_results.csv'))